In [1]:
import pandas as pd

#load the excel file
test_data = pd.ExcelFile('/home/lhowk/Downloads/gorilla_test_data.xlsx') 


#extact list of the sheets in data frames
meter_list=test_data.parse('meter_list')
forecast_table=test_data.parse('forecast_table')
rate_table=test_data.parse('rate_table')


#Creation of a table for prices
RealPriceAQ=pd.merge(meter_list, rate_table, how="left", on="exit_zone")

#Creation of column to filter aq_kwh below category
RealPriceAQ['Filter_Qty1']=RealPriceAQ['aq_kwh']-RealPriceAQ['aq_min_kwh']

#Creation of column to filter aq_kwh above category
RealPriceAQ['Filter_Qty2']=RealPriceAQ['aq_kwh']-RealPriceAQ['aq_max_kwh']

#filter to keep only the prices of the right category
RealPriceAQ=RealPriceAQ[(RealPriceAQ['Filter_Qty2']<0)&(RealPriceAQ['Filter_Qty1']>=0)]

#remove filters (2 last columns) and rename 'date' for clarity
RealPriceAQ=RealPriceAQ.iloc[: , :-2]
RealPriceAQ=RealPriceAQ.rename(columns = {'date':'PricePeriod'})


#merge of the forecast_table with the prices for each meter_id with the given AQ
aggregate=pd.merge(forecast_table, RealPriceAQ, how="left", on="meter_id")


#filter to keep only the rows that are in the right PricePeriod
#remove all rows with a date before the PricePeriod
aggregate=aggregate[aggregate['date']>=aggregate['PricePeriod']]
#keep for each meter_id and given date the most recent price period
idx=aggregate.groupby(['meter_id','date'])['PricePeriod'].transform(max) == aggregate['PricePeriod']
aggregate=aggregate[idx]

#create the array for price per day for each day and each meter
aggregate['PricebyDay']=aggregate['kwh']*aggregate['rate_p_per_kwh']

#summarize for each meter the total production
sumProdbyMeter=(aggregate.groupby(['meter_id'])['kwh'].sum()).map('{:,.0f}'.format)


#Summarize for each meter the total cost
totalCostbyMeter=aggregate.groupby(['meter_id'])['PricebyDay'].sum()
totalCostbyMeter=(totalCostbyMeter/100).map('£{:,.2f}'.format)

#merge to generate output
output = pd.merge(sumProdbyMeter, totalCostbyMeter, how="left", on="meter_id")
output.rename(columns={'meter_id': 'Meter ID', 'kwh': 'Total Estimated Consumption (kWh)', 'PricebyDay':'Total Cost (£)'}, inplace=True)
display(output)

,Total Estimated Consumption (kWh),Total Cost (£)
meter_id,,
14676236,"28,978",£100.15
34509937,"78,324",£275.49
50264822,"265,667",£731.24
88357331,"484,399","£1,433.16"


In [2]:
def randomGenerator(nbrMeter):
#Generates a dataframe of meter_id, size of the meter and exit_zone
#takes as input nbrMeter, the number of meter we want to generate
#The meter_id is an increment from 0 to nbrMeter
#Size is generated randomly and exit zone is picked randomly in the list for each meter
    
    import numpy as np
    import pandas as pd
    
    #list of possible exit_zones
    exit_zone=['EA1', 'EA2', 'EA3', 'EA4', 'EM1', 'EM2', 'EM3', 'EM4', 'LC', 'LO', 'LS', 'LT', 'LW',
    'NE1', 'NE2', 'NE3', 'NO1', 'NO2', 'NT1', 'NT2', 'NT3', 'NW1', 'NW2', 'SC1', 'SC2',
    'SC4', 'SE1', 'SE2', 'SO1', 'SO2', 'SW1', 'SW2', 'SW3', 'WA1', 'WA2', 'WM1', 'WM2',
    'WM3']

    #pick randomly the exit_zone based on list index
    exit_zone_index=np.random.randint(low = 0,high=len(exit_zone),size=nbrMeter)
    exit_zone_list=[exit_zone[i] for i in exit_zone_index]
    
    #Generate the size of the meters
    size_list=np.random.randint(low = 1,high=100000,size=nbrMeter)
    
    #attributes a incremental id_meter for each meter
    meter_id=range(nbrMeter)
    
    #create the dataframe for output
    rdnMeterList=pd.DataFrame({'meter_id':meter_id,'aq_kwh':size_list,'exit_zone':exit_zone_list})

    return(rdnMeterList)

randomGenerator(5)

,meter_id,aq_kwh,exit_zone
0,0,72047,EA1
1,1,42294,WM3
2,2,30005,WA1
3,3,81435,EM1
4,4,13884,SC1


In [3]:
def randomTimeseries(listMeters,startDate,nbrDays):
#Generates a dataframe of forecasted consumption
#Generated randomly, based on the list of meters, starting date (YYYY:MM:DD) and number of days

    from datetime import datetime, timedelta
    import pandas as pd
    import numpy as np
    
    #get startdate from parameters, and calculate the end date
    startDate=datetime.strptime(startDate, "%Y:%m:%d")
    endDate=startDate + timedelta(days=nbrDays)
    
    #timeRange with a one hour frequency, converted to pandas dataframe
    timeRange=pd.date_range(start=startDate, end=endDate, freq='1H')
    timeRange=timeRange.to_frame(index=False)
    
    #create an empty dataFrame to store all the meters data
    rdnTimeseries=pd.DataFrame()
    rdnTimeseries['meter_id']=[]
    rdnTimeseries['date']=[]
    rdnTimeseries['kwh']=[]
    
    #Loop for each meter to be generated
    #The dataframe is populated on each iteration of the loop with the time series
    for Meter in listMeters['meter_id']:
        #value of production generated randomly between 0 et 1000 
        rdnKwh=pd.DataFrame(np.around(np.random.uniform(low = 0,high=1000,size=len(timeRange)),decimals=4))
        newMeterData=pd.concat([pd.DataFrame(Meter*np.ones(len(timeRange))),timeRange,rdnKwh],axis=1)
        newMeterData.set_axis(["meter_id", "date", "kwh"], axis="columns", inplace=True)
        rdnTimeseries=pd.concat([rdnTimeseries,newMeterData], ignore_index=True)
    return(rdnTimeseries)

randomTimeseries(randomGenerator(5),'2022:01:1',500)

,meter_id,date,kwh
0,0.0,2022-01-01 00:00:00,769.6369
1,0.0,2022-01-01 01:00:00,990.1992
2,0.0,2022-01-01 02:00:00,179.4337
3,0.0,2022-01-01 03:00:00,197.7400
4,0.0,2022-01-01 04:00:00,199.8808
...,...,...,...
60000,4.0,2023-05-15 20:00:00,216.4850
60001,4.0,2023-05-15 21:00:00,55.9484
60002,4.0,2023-05-15 22:00:00,679.1322
60003,4.0,2023-05-15 23:00:00,399.5830


In [4]:
def transportRdnTable(meter_list, forecast_table, rate_table):
    import pandas as pd

    #Creation of a table for prices
    RealPriceAQ=pd.merge(meter_list, rate_table, how="left", on="exit_zone")

    #Creation of column to filter aq_kwh below category
    RealPriceAQ['Filter_Qty1']=RealPriceAQ['aq_kwh']-RealPriceAQ['aq_min_kwh']

    #Creation of column to filter aq_kwh above category
    RealPriceAQ['Filter_Qty2']=RealPriceAQ['aq_kwh']-RealPriceAQ['aq_max_kwh']

    #filter to keep only the prices of the right category
    RealPriceAQ=RealPriceAQ[(RealPriceAQ['Filter_Qty2']<0)&(RealPriceAQ['Filter_Qty1']>=0)]

    #remove filters (2 last columns) and rename 'date' for clarity
    RealPriceAQ=RealPriceAQ.iloc[: , :-2]
    RealPriceAQ=RealPriceAQ.rename(columns = {'date':'PricePeriod'})


    #merge of the forecast_table with the prices for each meter_id with the given AQ
    aggregate=pd.merge(forecast_table, RealPriceAQ, how="left", on="meter_id")


    #filter to keep only the rows that are in the right PricePeriod
    #remove all rows with a date before the PricePeriod
    aggregate=aggregate[aggregate['date']>=aggregate['PricePeriod']]
    #keep for each meter_id and given date the most recent price period
    idx=aggregate.groupby(['meter_id','date'])['PricePeriod'].transform(max) == aggregate['PricePeriod']
    aggregate=aggregate[idx]

    #create the array for price per day for each day and each meter
    aggregate['PricebyDay']=aggregate['kwh']*aggregate['rate_p_per_kwh']

    #summarize for each meter the total production
    sumProdbyMeter=(aggregate.groupby(['meter_id'])['kwh'].sum()).map('{:,.0f}'.format)


    #Summarize for each meter the total cost
    totalCostbyMeter=aggregate.groupby(['meter_id'])['PricebyDay'].sum()
    totalCostbyMeter=(totalCostbyMeter/100).map('£{:,.2f}'.format)

    #merge to generate output
    output = pd.merge(sumProdbyMeter, totalCostbyMeter, how="left", on="meter_id")
    output.rename(columns={'meter_id': 'Meter ID', 'kwh': 'Total Estimated Consumption (kWh)', 'PricebyDay':'Total Cost (£)'}, inplace=True)
    return(output)


def launchTransport(nbrMeters, nbrDays):
    import pandas as pd

    listMeterTest=randomGenerator(nbrMeters)

    forecastTest=randomTimeseries(listMeterTest,'2022:01:1',nbrDays)


    test_data = pd.ExcelFile('/home/lhowk/Downloads/gorilla_test_data.xlsx') 
    rate_table=test_data.parse('rate_table')


    return transportRdnTable(listMeterTest, forecastTest, rate_table)

launchTransport(10,100)

,Total Estimated Consumption (kWh),Total Cost (£)
meter_id,,
0.0,"1,196,436","£3,444.77"
1.0,"1,218,866","£4,517.70"
2.0,"1,206,765","£5,257.81"
3.0,"1,215,832","£5,081.08"
4.0,"1,197,501","£4,975.02"
5.0,"1,190,589","£4,148.19"
6.0,"1,188,555","£4,409.50"
7.0,"1,199,383","£3,570.43"
8.0,"1,214,426","£3,284.24"


In [12]:
import numpy as np
import perfplot

perfplot.save(
    "p.png",
    transparent=False,
    setup=lambda n: n,
    kernels=[
        lambda a: launchTransport(1, a),
        lambda a: launchTransport(a, 1)],
    n_range=[2 ** k for k in range(16)],
    labels=["nbrMeter", "nbrDays"],
    xlabel="len(a)",
    # More optional arguments with their default values:
     logx=False,  # set to True or False to force scaling
     logy=False,
    equality_check=None,  # set to None to disable "correctness" assertion
    show_progress=True,
)


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may 
help.

Overall ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--

Overall ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--
Kernels ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--

Overall ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--
Kernels ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--

Overall ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--
Kernels ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--

...

Overall ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━  94% 0:00:14
Kernels ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━  50% -:--:--

Overall ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━  94% 0:00:14
Kernels ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━  50% -:--:--

Overall ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━  94% 0:00:14
Kernels ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━  50% -:--:--

Overall ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━  94% 0:00:14
Kernels ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━  50% -:--:--

Overall ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Kernels ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--